In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import string
import matplotlib.pyplot as plt
import seaborn as sns
import re

from collections import Counter
from sklearn import ensemble
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer, normalize
from sklearn.cluster import KMeans, MiniBatchKMeans

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('C://duh/Thinkful/Thinkful/Exercises/Medium_AggregatedData.csv')

In [3]:
df.head()

,audioVersionDurationSec,codeBlock,codeBlockCount,collectionId,createdDate,createdDatetime,firstPublishedDate,firstPublishedDatetime,imageCount,isSubscriptionLocked,...,slug,name,postCount,author,bio,userId,userName,usersFollowedByCount,usersFollowedCount,scrappedDate
0,0,NaN,0.0,638f418c8464,2018-09-18,2018-09-18 20:55:34,2018-09-18,2018-09-18 20:57:03,1,False,...,blockchain,Blockchain,265164.0,Anar Babaev,NaN,f1ad85af0169,babaevanar,450.0,404.0,20181104
1,0,NaN,0.0,638f418c8464,2018-09-18,2018-09-18 20:55:34,2018-09-18,2018-09-18 20:57:03,1,False,...,samsung,Samsung,5708.0,Anar Babaev,NaN,f1ad85af0169,babaevanar,450.0,404.0,20181104
2,0,NaN,0.0,638f418c8464,2018-09-18,2018-09-18 20:55:34,2018-09-18,2018-09-18 20:57:03,1,False,...,it,It,3720.0,Anar Babaev,NaN,f1ad85af0169,babaevanar,450.0,404.0,20181104
3,0,NaN,0.0,NaN,2018-01-07,2018-01-07 17:04:37,2018-01-07,2018-01-07 17:06:29,13,False,...,technology,Technology,166125.0,George Sykes,NaN,93b9e94f08ca,tasty231,6.0,22.0,20181104
4,0,NaN,0.0,NaN,2018-01-07,2018-01-07 17:04:37,2018-01-07,2018-01-07 17:06:29,13,False,...,robotics,Robotics,9103.0,George Sykes,NaN,93b9e94f08ca,tasty231,6.0,22.0,20181104


In [4]:
df = df[['text', 'title', 'author', 'language']]
df.drop(df.index[df.language != 'en'], inplace=True)
df = df.reset_index(drop=True)
authorindex = []
for author in df.author.unique():
    if df.text.where(df.author == author).nunique() >= 100 and df.text.where(df.author == author).nunique() <= 150:
        authorindex.append(author)
df = df.query('author in @authorindex')
df = df.reset_index(drop=True)
nlp = spacy.load('en')
df['doc'] = None
df['bow'] = None

del authorindex

sentences = pd.DataFrame()

In [5]:
df.head()

,text,title,author,language,doc,bow
0,It’s Pretty Bleeping Scary.\nAnd this is Sam H...,It’s Pretty Bleeping Scary.,Peter Marshall,en,None,None
1,It’s Pretty Bleeping Scary.\nAnd this is Sam H...,It’s Pretty Bleeping Scary.,Peter Marshall,en,None,None
2,It’s Pretty Bleeping Scary.\nAnd this is Sam H...,It’s Pretty Bleeping Scary.,Peter Marshall,en,None,None
3,Webography for 4 dummies to make it in machine...,Webography for 4 dummies to make it in machine...,WELTARE Strategies,en,None,None
4,Webography for 4 dummies to make it in machine...,Webography for 4 dummies to make it in machine...,WELTARE Strategies,en,None,None


In [6]:
df.drop_duplicates(subset='text', inplace=True)
df = df.reset_index(drop=True)

In [7]:
df.shape

(1174, 6)

In [8]:
for x in range(len(df.text)):
    if x % 250 == 0:
        print("Processing row {}".format(x))
    #df.text[x] = re.sub(df.title[x], '', df.text[x])
    df.text[x] = df.text[x].lower()
    df.text[x] = re.sub('https://*.*?\\n', ' ', df.text[x])
    df.text[x] = re.sub('http://*.*?\\n', ' ', df.text[x])
    df.text[x] = re.sub('\\n', ' ', df.text[x])
    df.text[x] = re.sub('bibliography.*', '', df.text[x])
    df.text[x] = re.sub('[%s]' % re.escape(string.punctuation), '', df.text[x])
    df.text[x] = re.sub('\w*\d\w*', '', df.text[x])
    df.text[x] = re.sub('’', '', df.text[x])
    df.text[x] = re.sub('—', ' ', df.text[x])
    df.text[x] = re.sub('”', '', df.text[x])
    df.text[x] = re.sub('“', '', df.text[x])
    df.text[x] = re.sub('‘', '', df.text[x])
    df.text[x] = re.sub('€', '', df.text[x])
    df.text[x] = re.sub('…', ' ', df.text[x])
    df.text[x] = re.sub('\ \ ', ' ', df.text[x])
    df.text[x] = re.sub('quest ce quune classe concrete google search févr la figure suivante représente nos classes classe animal nous avons bien notre classe mère animal et wwwgoogleie les décorateurs python est un langage de programmation qui nécessite décrire moins de lignes de code que le c ou le c il se veut openclassroomscom objet est une instance de classe google search en programmation orientée objet on appelle instance dune classe un objet avec un comportement et un état tous deux wwwgoogleie classe abstraite google search classe abstraite et interface quest ce quune classe abstraite une classe abstraite est une classe incomplète elle wwwgoogleie creer une classe python google search nov nous allons aussi essayer de comprendre les mécanismes de la programmation orientée objet en python wwwgoogleie première approche des classes python est un langage de programmation qui nécessite décrire moins de lignes de code que le c ou le c il se veut openclassroomscom', '', df.text[x])
    df.text[x] = df.text[x].strip()
    if df.text[x] == '':
        df.drop([x], inplace=True)
        df = df.reset_index(drop=True)

Processing row 0
Processing row 250
Processing row 500
Processing row 750
Processing row 1000


In [9]:
df.head()

,text,title,author,language,doc,bow
0,its pretty bleeping scary and this is sam harr...,It’s Pretty Bleeping Scary.,Peter Marshall,en,None,None
1,webography for dummies to make it in machine l...,Webography for 4 dummies to make it in machine...,WELTARE Strategies,en,None,None
2,technological unemployment and ubi reutersmura...,Technological Unemployment and UBI,Michael K. Spencer,en,None,None
3,vantagepoint forex weekly outlook for march t...,VantagePoint Forex Weekly Outlook for March 5t...,Vantagepoint ai,en,None,None
4,humanintheloop ai for emergency response more ...,Human-in-the-Loop AI for Emergency Response & ...,TWiML & AI,en,None,None


In [10]:
df.shape

(1174, 6)

In [11]:
for x in range(len(df.text)):
    df.doc[x] = nlp(df.text[x])
    sents = [[sent, df.author[x]] for sent in df.doc[x].sents]
    sentences = sentences.append(sents, ignore_index=True)

In [12]:
sentences.head()

,0,1
0,"(its, pretty, bleeping, scary, and, this, is, ...",Peter Marshall
1,"(and, i, think, he, s, absolutely, right, with...",Peter Marshall
2,"(webography, for, dummies, to, make, it, in, m...",WELTARE Strategies
3,"(what, s, the, difference, with, docker, conta...",WELTARE Strategies
4,"(and, now, i, m, trying, to, create, a, text, ...",WELTARE Strategies


In [13]:
sentences.shape

(11188, 2)

In [14]:
def bag_of_words(text):
    
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

def bow_features(sentences, common_words):
    
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    for i, sentence in enumerate(df['text_sentence']):
        
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        for word in words:
            df.loc[i, word] += 1
        
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

In [15]:
for x in range(len(df.text)):
    df.bow[x] = bag_of_words(df.doc[x])

In [16]:
df.head()

,text,title,author,language,doc,bow
0,its pretty bleeping scary and this is sam harr...,It’s Pretty Bleeping Scary.,Peter Marshall,en,"(its, pretty, bleeping, scary, and, this, is, ...","[think, pretty, bleeping, scary, agi, absolute..."
1,webography for dummies to make it in machine l...,Webography for 4 dummies to make it in machine...,WELTARE Strategies,en,"(webography, for, dummies, to, make, it, in, m...","[create, file, touch, powershell, linux, langu..."
2,technological unemployment and ubi reutersmura...,Technological Unemployment and UBI,Michael K. Spencer,en,"(technological, unemployment, and, ubi, reuter...","[technological, human, amazon, world, robot, u..."
3,vantagepoint forex weekly outlook for march t...,VantagePoint Forex Weekly Outlook for March 5t...,Vantagepoint ai,en,"(vantagepoint, forex, weekly, outlook, for, ma...","[level, key, vantagepoint, market, pair, movem..."
4,humanintheloop ai for emergency response more ...,Human-in-the-Loop AI for Emergency Response & ...,TWiML & AI,en,"(humanintheloop, ai, for, emergency, response,...","[figure, ai, twiml, machine, learning, rob, le..."


In [17]:
common_words = set()
for x in range(len(df.text)):
    common_words.update(df.bow[x])

In [18]:
word_counts = pd.DataFrame()
a = pd.from_pickle('C:\\duh\Thinkful\Thinkful\Exercises\a.pkl')
b = pd.from_pickle('C:\\duh\Thinkful\Thinkful\Exercises\b.pkl')
c = pd.from_pickle('C:\\duh\Thinkful\Thinkful\Exercises\c.pkl')
d = pd.from_pickle('C:\\duh\Thinkful\Thinkful\Exercises\d.pkl')

word_counts = word_counts.append([a, b, c, d], ignore_index=True)

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300


KeyboardInterrupt: 

In [ ]:
word_counts.head()

In [ ]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0,
                                                    stratify=Y)

In [ ]:
#X_norm = normalize(X)

#pca = PCA(.95)
#X_pca = pca.fit_transform(X_norm)

svd= TruncatedSVD(10)
lsa = make_pipeline(svd, Normalizer(copy=False))
# Run SVD on the training data, then project the training data.
X_lsa = lsa.fit_transform(X)

# Calculate predicted values.
y_pred = KMeans(random_state=0).fit_predict(X_lsa)

In [ ]:
variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components:",total_variance*100)

In [ ]:
plt.figure(figsize=(20,10))
plt.scatter(X_lsa[:, 0], X_lsa[:, 1], c=word_counts.text_source.drop_duplicates())
plt.scatter(X_lsa[:, 2], X_lsa[:, 3], c=y_pred)
plt.scatter(X_lsa[:, 4], X_lsa[:, 5], c=y_pred)
plt.scatter(X_lsa[:, 6], X_lsa[:, 7], c=y_pred)
plt.scatter(X_lsa[:, 8], X_lsa[:, 9], c=y_pred)
plt.legend()
plt.show()

In [ ]:
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
        
print('Training score: ', lr.score(X_train, y_train))
print('Test score: ', lr.score(X_test, y_test))

In [ ]:
svm = SVC(kernel='linear')
train = svm.fit(X_train, y_train)

print('Training score: ', svm.score(X_train, y_train))
print('Test score: ', svm.score(X_test, y_test))